In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,roc_auc_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTENC
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,OrdinalEncoder,TargetEncoder

In [13]:
df=pd.read_csv("auto insurance.csv")


In [14]:
df['policy_bind_date'] = pd.to_datetime(df['policy_bind_date'])  
df['policy_year'] = df['policy_bind_date'].dt.year

df['incident_date'] = pd.to_datetime(df['incident_date'])  
df['incident_year'] = df['incident_date'].dt.year

df['year_diff'] = df['incident_year']-df['policy_year']



In [15]:
df=df.drop(["insured_hobbies","incident_date","policy_bind_date","incident_location","policy_number","_c39"],axis=1)
df.shape

(1000, 37)

In [ ]:

x=df.drop(["fraud_reported"],axis=1)
y=df["fraud_reported"]


categorical_cols = df.select_dtypes(include=['object', 'category'])
categorical_indices = [x.columns.get_loc(col) for col in categorical_cols.columns if col in x.columns]

# Apply SMOTENC
smote_nc = SMOTENC(categorical_features=categorical_indices, random_state=42)
x, y = smote_nc.fit_resample(x, y)


d:\anaconda\lib\site-packages\sklearn\base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
d:\anaconda\lib\site-packages\sklearn\base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


In [19]:
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.2,random_state=42,stratify=y)

In [ ]:
# ---------- Column Groups ----------
freq_columns = ["policy_state", "policy_csl", "incident_type", "insured_occupation",
                "insured_relationship", "authorities_contacted", "incident_state", "incident_city"]

ohe_columns = ["insured_sex", "property_damage", "police_report_available", "incident_type", "collision_type"]

te_columns = ["auto_make", "auto_model", "insured_zip"]

ordinal_columns = {
    'insured_education_level': ['JD', 'High School', 'Associate', 'MD', 'Masters', 'PhD', 'College'],
    'incident_severity': ['Trivial Damage', 'Minor Damage', 'Major Damage', 'Total Loss']
}

scale_columns = ["policy_deductable", "policy_annual_premium", "total_claim_amount",
                 "injury_claim", "property_claim", "vehicle_claim", "umbrella_limit",
                 "capital-gains", "capital-loss"]


le = LabelEncoder()
df["fraud_reported"] = le.fit_transform(df["fraud_reported"])
y = df["fraud_reported"]

# ---------- ColumnTransformer ----------
preprocessor = ColumnTransformer(
    transformers=[
        ("freq_enc", ce.CountEncoder(normalize=True), freq_columns),
        ("target_enc", TargetEncoder(), te_columns),
        ("ohe", OneHotEncoder(sparse_output=True, handle_unknown='ignore'), ohe_columns),
        ("ordinal_edu", OrdinalEncoder(categories=[ordinal_columns["insured_education_level"]]), ["insured_education_level"]),
        ("ordinal_sev", OrdinalEncoder(categories=[ordinal_columns["incident_severity"]]), ["incident_severity"]),
        ("scaler", StandardScaler(), scale_columns)
    ],
    remainder="drop"  # drop unused columns
)


model = RandomForestClassifier(
    n_estimators=300,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    max_depth=40,
    bootstrap=True,
    random_state=42
)



# ---------- Pipeline ----------
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ('classifier', model)
])




In [24]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('freq_enc',
                                                  CountEncoder(combine_min_nan_groups=True,
                                                               normalize=True),
                                                  ['policy_state', 'policy_csl',
                                                   'incident_type',
                                                   'insured_occupation',
                                                   'insured_relationship',
                                                   'authorities_contacted',
                                                   'incident_state',
                                                   'incident_city']),
                                                 ('target_enc', TargetEncoder(),
                                                  ['auto_make', 'auto_model',
                                                   'insured_zip'...
                                                                              'Minor '
                                                                              'Damage',
                                                                              'Major '
                                                                              'Damage',
                                                                              'Total '
                                                                              'Loss']]),
                                                  ['incident_severity']),
                                                 ('scaler', StandardScaler(),
                                                  ['policy_deductable',
                                                   'policy_annual_premium',
                                                   'total_claim_amount',
                                                   'injury_claim',
                                                   'property_claim',
                                                   'vehicle_claim',
                                                   'umbrella_limit',
                                                   'capital-gains',
                                                   'capital-loss'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=40, n_estimators=300,
                                        random_state=42))])

In [28]:
from sklearn.metrics import classification_report, roc_auc_score


y_pred = pipeline.predict(X_test)


y_proba = pipeline.predict_proba(X_test)[:, 1]  


print(" Classification Report:")
print(classification_report(y_test, y_pred))


roc_auc = roc_auc_score(y_test, y_proba)
print(f" ROC AUC Score: {roc_auc:.4f}")


 Classification Report:
              precision    recall  f1-score   support

           N       0.84      0.85      0.85       151
           Y       0.85      0.83      0.84       151

    accuracy                           0.84       302
   macro avg       0.84      0.84      0.84       302
weighted avg       0.84      0.84      0.84       302

 ROC AUC Score: 0.9275


In [29]:
import pickle

with open("random_forest_pipeline.pkl", "wb") as f:
    pickle.dump(pipeline, f)


In [33]:
x.to_csv("example.csv")